In [5]:
import pandas as pd
import json
import re
import numpy as np
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [6]:
# 파일 경로 설정
file_path1 = r'국토교통부_법령.json'
file_path2 = r'법령정보_0412.json'
file_path3 = r'법령데이터_20240726.json'

In [7]:
# JSON 파일 로드 함수
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 시행일자 추출 함수
def extract_effective_date(content):
    content = re.sub(r'\xa0', ' ', content)
    match = re.search(r"시행 (\d{4}\.\s\d+\.\s\d+)", content)
    return match.group(1) if match else None

# 법령 번호 추출 함수
def extract_law_number(content):
    content = re.sub(r'\xa0', ' ', content)
    match = re.search(r'\w+ 제\d+호', content)
    return match.group(0).split(' ')[-1] if match else None

# 법령 종류+번호 추출 함수
def extract_law_id(content):
    content = re.sub(r'\xa0', ' ', content)
    match = re.search(r'\w+ 제\d+호', content)
    return match.group(0) if match else None

# '장' 추출 함수
def extract_chapters(content):
    chapters = {}
    chapter_splits = re.split(r'\n\s{8}(제\d+장.*?)\n', content)
    
    for i in range(1, len(chapter_splits), 2):
        chapter_title = chapter_splits[i].strip()
        chapter_content = chapter_splits[i + 1]
        chapters[chapter_title] = chapter_content
    return chapters

def clean_addenda_title(title):
    return re.sub(r'부\s{2,}칙', '부칙', title)

def clean_text(text):
    # 줄바꿈과 여러 공백을 단일 공백으로 정리
    return re.sub(r'\n\s*', '', text).strip()
    
def extract_addenda(content):
    # '\xa0' 제거
    content = re.sub(r'\xa0', '', content)
    
    # 부칙 전체 추출 ('부      칙' 형태 포함)
    matches = re.findall(r'(\n\s*부칙.*?)(?=\n\s*부칙|$)', content, re.DOTALL)
    addenda = {}
    
    for match in matches:
        # 첫 줄의 부칙 제목과 내용을 나누고 제목의 공백 정리
        parts = match.strip().split('\n', 1)
        addenda_title = clean_addenda_title(parts[0].strip())  # 제목 공백 정리
        addenda_content = '\n       ' + parts[1] if len(parts) > 1 else None
        
        # 부칙 내용을 조항별로 분리하여 각 조항 제목을 key, 내용을 value로 저장
        addenda_dict = {}
        if addenda_content:
            # 조항을 '제\d조(내용)' 패턴으로 분리하여 키-값 저장
            items = re.split(r'(?=\n\s*제\d+조\s*\([^)]+\))', addenda_content)
            if len(items) > 1:  # 조항 패턴이 있으면 각 조항을 분리해서 저장
                for item in items:
                    if item.strip():
                        match = re.match(r'\n\s*(제\d+조\s*\([^)]+\))\s*(.*)', item, re.DOTALL)
                        if match:
                            clause_title = match.group(1).strip()
                            clause_content = match.group(2).strip()
                            addenda_dict[clause_title] = clean_text(clause_content)
            else:  # 조항 패턴이 없으면 전체 내용을 하나의 항목으로 저장
                addenda_dict["내용"] = clean_text(addenda_content).strip()
        
        # addenda_title을 키로, 조항 딕셔너리인 addenda_dict를 값으로 설정
        addenda[addenda_title] = addenda_dict if addenda_dict else None

    # 본문에서 부칙을 제외하고 메인 내용만 추출
    main_content = re.sub(r'(\n\s*부칙.*?)(?=\n\s*부칙|$)', '', content, flags=re.DOTALL).strip()
    
    return main_content, addenda
def split_only(text):
    # 조항 패턴: '제\d{1,2}조' 또는 '제\d{1,2}조의?\d*'
    pattern = r'(\n\s*제\d{1,3}조의?\d*\s?\(.*?\))|(\n\s*제\d{1,2}조\s?\(.*?\))'
    parts = re.split(pattern, text)
    
    # '제\d+조' 패턴이 없는 경우 원래 텍스트 반환
    if len(parts) == 1:
        return [text.strip()]

    # 결과를 저장 생성
    result = {}
    for i in range(1, len(parts), 3):
        part_title = parts[i] if parts[i] else parts[i + 1] if i + 1 < len(parts) else None
        if part_title:
            part_title = part_title.strip()
        else:
            continue

        part_content = parts[i + 2] if i + 2 < len(parts) else None
        if part_content:
            result[part_title] = part_content.strip()
        else:
            result[part_title] = ''
    
    return result

# 별지 처리 함수
def extract_appendix(content):
    if '[별지' in content:
        appendix_content = content.split('[별지', 1)[1].strip()
        main_content = content.split('[별지', 1)[0].strip()
        return main_content, '[별지' + appendix_content
    else:
        return content, None


In [8]:
data1 = load_json(file_path1)
data2 = load_json(file_path2)
data3 = load_json(file_path3)


# 데이터 프레임 생성
df1 = pd.DataFrame(data1['result'])
df2 = pd.DataFrame(data2['result'])
df3 = pd.DataFrame(data3['result'])


# 데이터 병합 및 중복 제거
df = pd.concat([df1, df2, df3], ignore_index=True).drop_duplicates()

In [9]:
data1 = load_json(file_path1)
data2 = load_json(file_path2)
data3 = load_json(file_path3)


# 데이터 프레임 생성
df1 = pd.DataFrame(data1['result'])
df2 = pd.DataFrame(data2['result'])
df3 = pd.DataFrame(data3['result'])


# 데이터 병합 및 중복 제거
df = pd.concat([df1, df2, df3], ignore_index=True).drop_duplicates()

# 별지 처리
df[['content', 'appendix']] = df['content'].apply(lambda x: pd.Series(extract_appendix(x)))

# 부칙 처리
df[['content', 'addenda']] = df['content'].apply(lambda x: pd.Series(extract_addenda(x)))

df['effective_date'] = df['content'].apply(extract_effective_date)
df['law_number'] = df['content'].apply(extract_law_number)
df['law_id'] = df['content'].apply(extract_law_id)
df['chapters'] = df['content'].apply(extract_chapters)

# 챕터가 없는 경우 '제0장'으로 설정
df['chapters'] = df.apply(lambda row: {'제0장': row['content']} if len(row['chapters']) == 0 else row['chapters'], axis=1)

df['only_chapters'] = df['chapters']

# 문자열 정리
df['title'] = df['title'].str.strip()
df['law_number'] = df['law_number'].str.strip()
df['law_id'] = df['law_id'].str.strip()

# 챕터 데이터에 대해 조항 분리 및 정리 적용
df['chapters'] = df['chapters'].apply(lambda chapters: {k: split_only(v) for k, v in chapters.items()})

In [16]:
df['chapters'][174]

{'제1장 총칙': {'제1조(목적)': '이 영은 「건축법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다. \n        [전문개정 2008. 10. 29.]',
  '제2조(정의)': '이 영에서 사용하는 용어의 뜻은 다음과 같다. <개정 2009. 7. 16., 2010. 2. 18., 2011. 12. 8., 2011. 12. 30., 2013. 3. 23., 2014. 11. 11., 2014. 11. 28., 2015. 9. 22., 2016. 1. 19., 2016. 5. 17., 2016. 6. 30., 2016. 7. 19., 2017. 2. 3., 2018. 9. 4., 2020. 4. 28.> \n         \n       1. “신축”이란 건축물이 없는 대지(기존 건축물이 해체되거나 멸실된 대지를 포함한다)에 새로 건축물을 축조(築造)하는 것[부속건축물만 있는 대지에 새로 주된 건축물을 축조하는 것을 포함하되, 개축(改築) 또는 재축(再築)하는 것은 제외한다]을 말한다.  \n       2. “증축”이란 기존 건축물이 있는 대지에서 건축물의 건축면적, 연면적, 층수 또는 높이를 늘리는 것을 말한다.  \n       3. “개축”이란 기존 건축물의 전부 또는 일부[내력벽ㆍ기둥ㆍ보ㆍ지붕틀(제16호에 따른 한옥의 경우에는 지붕틀의 범위에서 서까래는 제외한다) 중 셋 이상이 포함되는 경우를 말한다]를 해체하고 그 대지에 종전과 같은 규모의 범위에서 건축물을 다시 축조하는 것을 말한다.  \n       4. “재축”이란 건축물이 천재지변이나 그 밖의 재해(災害)로 멸실된 경우 그 대지에 다음 각 목의 요건을 모두 갖추어 다시 축조하는 것을 말한다.  \n       가. 연면적 합계는 종전 규모 이하로 할 것  \n       나. 동(棟)수, 층수 및 높이는 다음의 어느 하나에 해당할 것 \n       1) 동수, 층수 및 높이가 모두 종전 규모 이하일 것 \n       2) 동수, 층수 또는 높이의 어느 

In [17]:
print(data1['result'][174]['content'])

 
      
      건축법 시행령  
      
      [시행 2024. 9. 13.] [대통령령 제33717호, 2023. 9. 12., 일부개정]  
      
     
      
       국토교통부(건축정책과 - 건축제도 일반), 044-201-3762, 3763  
       국토교통부(건축안전과 - 건축구조 규정 운영), 044-201-4991  
       국토교통부(녹색건축과 - 건축설비ㆍ조경 규정 운영), 044-201-4753  
       국토교통부(건축안전과 - 피난ㆍ마감재료 규정 운영), 044-201-4992  
       국토교통부(건축정책과 - 건축감리 규정 운영), 044-201-4752  
       국토교통부(건축정책과 - 위반건축물 규정 운영), 044-201-3762, 3761  
      
      
      
      
               제1장 총칙  
      
      
      
       
          
          
       
       
          제1조(목적)   이 영은 「건축법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다. 
        [전문개정 2008. 10. 29.]  
        
       
      
      
      
       
          
          
       
       
          제2조(정의)   이 영에서 사용하는 용어의 뜻은 다음과 같다. <개정 2009. 7. 16., 2010. 2. 18., 2011. 12. 8., 2011. 12. 30., 2013. 3. 23., 2014. 11. 11., 2014. 11. 28., 2015. 9. 22., 2016. 1. 19., 2016. 5. 17., 2016. 6. 30., 2016. 7. 19., 2017. 2. 3., 2018. 9. 4., 2020. 4. 28.> 
         
 

In [18]:
# NaN을 None으로 처리
df = df.replace({np.nan: None})

# 필요한 열만 선택
out = df[['menu', 'title', 'law_id', 'effective_date', 'chapters', 'addenda', 'appendix', 'url']]

# 데이터 그룹화 및 딕셔너리 변환
out = out.groupby('title', as_index=True).agg(list).to_dict('index')
out_ = {outer_key: {inner_key: (inner_value[0] if pd.notna(inner_value[0]) else None) for inner_key, inner_value in outer_value.items()} for outer_key, outer_value in out.items()}

# JSON 파일로 저장
with open('law_preprocessing.json', 'w', encoding="utf-8") as fp:
    json.dump(out_, fp, ensure_ascii=False, indent=4, allow_nan=False)

In [19]:
new_rows = []

for title, data in out_.items():
    law_id = data.get('law_id', '')
    effective_date = data.get('effective_date', '')
    
    for chapter_key, chapter_value in data['chapters'].items():
        chapter_content = ""
        
        if isinstance(chapter_value, dict):
            for article_key, article_value in chapter_value.items():
                if isinstance(article_value, dict):
                    article_content = " ".join([f"{sub_value}" for sub_key, sub_value in article_value.items()])
                    chapter_content += f" {article_key} : {article_content} ;"
                else:
                    chapter_content += f" {article_key} : {article_value} ; "
        else:
            chapter_content = chapter_value
        title = title.replace(' ', '')
        new_row = f"{title} ; {chapter_content.strip() if isinstance(chapter_content, str) else chapter_content}"
        new_rows.append(new_row)
new_df = pd.DataFrame(new_rows, columns=['passage'])
# new_df.to_csv('./law_passage_chapter.csv', encoding='utf-8-sig')

In [20]:
def article_preprocessing(data):
    article_row = []
    for i in range(len(data)):
        article = data['passage'][i].split(' ; ')
        for j in range(1, len(article)):
            article_row.append(f'{article[0]} ; {article[j].strip()}')
    index = [x for x in range(len(article_row))]
    article_data = pd.DataFrame(article_row, columns=['passage']).reset_index(drop=True)
    article_data['index'] = index
    return article_data

In [21]:
article_data = article_preprocessing(new_df)
article_data['passage'] = article_data['passage'].apply(clean_text)


In [22]:
article_data['law_name'] = article_data['passage'].apply(lambda x: x.split(';')[0].strip())
article_data['legal'] = article_data['passage'].apply(lambda x: x.split(' : ')[0])
article_data['legal'] = article_data['legal'].apply(lambda x: x.replace(';', ''))
article_data['docid_legal_content'] = article_data.apply(lambda x: f"{x.name} ; {x['passage']}", axis=1)

In [23]:
article_data.to_csv('../law_data/law_corpus.csv', encoding='utf-8-sig', lineterminator='\n')